In [1]:
!pip install ipywidgets
!pip install huggingface_hub
from huggingface_hub import notebook_login

notebook_login()


In [2]:
!pip install sentencepiece
!pip install tensorflow-datasets
!pip install transformers  datasets
!pip install py7zr
!pip install rouge_score
!pip install evaluate
!pip install rouge-score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.3/412.3 kB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.8/138.8 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.8/49.8 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 71.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 10.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24954 sha256=fbd2fd32ffb113597c67294d8e1822a16d9444dfe496f510b14e6c2e605cffad
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 3.0 MB/s eta 0:00:00


In [3]:
import pathlib
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [4]:
!pip install transformers==4.6.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 53.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 21.5 MB/s eta 0:00:00
  Installing build dependencies ... - \ | / - \ done
  Getting requirements to build wheel ... - done
  Preparing metadata (pyproject.toml) ... - done
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [51 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib.linux-x86_64-cpython-310
      creating build/lib.linux-x86_64-cpython-310/tokenizers
      copying py_src/tokenizers/__init__.py -> build/lib.linux-x86_64-cpython-310/tokenizers
      creating build/lib.linux-x86_64-cpython-310/tokenizers/models
      copying py_sr

In [5]:
# from transformers import pipeline, set_seed
from datasets import load_dataset
# import tensorflow as tf

In [6]:
import random
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    AdamW,
    get_scheduler,
    set_seed,
)

from datasets import load_dataset
from rouge_score import rouge_scorer

In [7]:
# !pip install py7zr

from rouge_score import rouge_scorer
seed_value = 42
random.seed(seed_value)
np.random.seed(seed_value)
torch.manual_seed(seed_value)
torch.cuda.manual_seed_all(seed_value)

rouge_scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

dataset = load_dataset("bakhitovd/ML_arxiv")

train_indices = list(range(len(dataset['train'])))
random.shuffle(train_indices)
train_dataset = dataset['train'].select(train_indices[:1000])

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/bakhitovd--ML_arxiv-f6e278e922eb0012/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [8]:
test_indices = list(range(len(dataset['test'])))
random.shuffle(test_indices)
test_dataset = dataset['test'].select(test_indices[:500])

checkpoint = "google/pegasus-wikihow"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
prefix = "summarize: "

In [9]:
import re

class SummaryDataset(Dataset):
    def __init__(self, dataset, tokenizer, prefix):
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.prefix = prefix

    def preprocess_text(self, text):

        text = re.sub(r'\$.*?\$', '', text)


        text = re.sub(r'\d+', '', text)


        text = re.sub(r'\s+', ' ', text)

        return text.strip()

    def __getitem__(self, idx):
        example = self.dataset[idx]
        article = example['article']
        abstract = example['abstract']


        article = self.preprocess_text(article)
        abstract = self.preprocess_text(abstract)

        inputs = self.prefix + article
        labels = self.prefix + abstract

        encoder_inputs = self.tokenizer(
            inputs,
            padding="max_length",
            truncation=True,
            max_length=512,
            return_tensors="pt"
        )
        decoder_inputs = self.tokenizer(
            labels,
            padding="max_length",
            truncation=True,
            max_length=128,
            return_tensors="pt"
        )

        input_ids = encoder_inputs.input_ids.squeeze()
        attention_mask = encoder_inputs.attention_mask.squeeze()
        decoder_input_ids = decoder_inputs.input_ids.squeeze()
        decoder_attention_mask = decoder_inputs.attention_mask.squeeze()
        labels = decoder_inputs.input_ids.squeeze()

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "decoder_input_ids": decoder_input_ids,
            "decoder_attention_mask": decoder_attention_mask,
            "labels": labels,
            "article": article,
        }

    def __len__(self):
        return len(self.dataset)
      

In [10]:
train_dataset = SummaryDataset(train_dataset, tokenizer, prefix)
test_dataset = SummaryDataset(test_dataset, tokenizer, prefix)

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)


def collate_fn(batch):
    input_ids = [item["input_ids"] for item in batch]
    attention_mask = [item["attention_mask"] for item in batch]
    decoder_input_ids = [item["decoder_input_ids"] for item in batch]
    decoder_attention_mask = [item["decoder_attention_mask"] for item in batch]
    labels = [item["labels"] for item in batch]
    article = [item["article"] for item in batch]

    return {
        "input_ids": torch.stack(input_ids),
        "attention_mask": torch.stack(attention_mask),
        "decoder_input_ids": torch.stack(decoder_input_ids),
        "decoder_attention_mask": torch.stack(decoder_attention_mask),
        "labels": torch.stack(labels),
        "article": article,
    }
train_loader = DataLoader(
    train_dataset,
    batch_size=4,
    shuffle=True,
    num_workers=2,
    collate_fn=collate_fn
)

test_loader = DataLoader(
    test_dataset,
    batch_size=4,
    shuffle=False,
    num_workers=2,
    collate_fn=collate_fn
)


model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = AdamW(model.parameters(), lr=1e-5, weight_decay=0.01)
total_steps = len(train_loader) * 1  # Adjust the number of epochs here
scheduler = get_scheduler("linear", optimizer, num_warmup_steps=0, num_training_steps=total_steps)

model.train()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


PegasusForConditionalGeneration(
  (model): PegasusModel(
    (shared): Embedding(96103, 1024, padding_idx=0)
    (encoder): PegasusEncoder(
      (embed_tokens): Embedding(96103, 1024, padding_idx=0)
      (embed_positions): PegasusSinusoidalPositionalEmbedding(512, 1024)
      (layers): ModuleList(
        (0-15): 16 x PegasusEncoderLayer(
          (self_attn): PegasusAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_nor

In [11]:
from rouge_score import rouge_scorer
from datasets import load_metric

# Create Rouge scorer
rouge_scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Load Rouge metric
rouge_metric = load_metric("rouge")

# Set the model to evaluation mode
model.eval()

# Initialize lists to store references and predictions
references = []
predictions = []

# Iterate over the test dataset
for batch in test_loader:
    # Move the batch to the device
    batch = {key: value.to(device) for key, value in batch.items() if isinstance(value, torch.Tensor)}

    # Disable gradient calculation
    with torch.no_grad():
        # Forward pass
       outputs = model.generate(
    input_ids=batch["input_ids"],
    attention_mask=batch["attention_mask"],
    decoder_input_ids=batch["decoder_input_ids"],
    decoder_attention_mask=batch["decoder_attention_mask"],
    max_new_tokens=128,  # Set the maximum number of generated tokens
   )


    # Decode the generated output
    decoded_outputs = [tokenizer.decode(ids, skip_special_tokens=True) for ids in outputs]
    decoded_targets = [tokenizer.decode(ids, skip_special_tokens=True) for ids in batch["labels"]]

    # Append references and predictions
    references.extend(decoded_targets)
    predictions.extend(decoded_outputs)

# Calculate Rouge scores
rouge_scores = rouge_metric.compute(predictions=predictions, references=references)

# Print the Rouge scores
print(rouge_scores)

{'rouge1': AggregateScore(low=Score(precision=0.8938847391400931, recall=0.9998922101449276, fmeasure=0.9253981874874467), mid=Score(precision=0.9124795429945901, recall=0.9999574275362318, fmeasure=0.9389683994794935), high=Score(precision=0.928949249520661, recall=1.0, fmeasure=0.9510008496673898)), 'rouge2': AggregateScore(low=Score(precision=0.891763275207213, recall=0.9998919606709079, fmeasure=0.9237194098480672), mid=Score(precision=0.9111699189212168, recall=0.999956969346443, fmeasure=0.938011431456026), high=Score(precision=0.9294395801755747, recall=1.0, fmeasure=0.950863922798724)), 'rougeL': AggregateScore(low=Score(precision=0.8940466522365387, recall=0.9998930932971014, fmeasure=0.9253878257744083), mid=Score(precision=0.9112601660693238, recall=0.9999583333333333, fmeasure=0.9381722673564717), high=Score(precision=0.928415561948102, recall=1.0, fmeasure=0.9503189194332711)), 'rougeLsum': AggregateScore(low=Score(precision=0.893720559489712, recall=0.9998922101449275, fm

In [12]:
# from transformers import AutoTokenizer
# import torch
# import itertools
# tokenizer = AutoTokenizer.from_pretrained('google/pegasus-wikihow')

# for epoch in range(1):
#     for batch in train_loader:
       
#             batch['article'] = [tokenizer.encode(text, padding='max_length', truncation=True, max_length=512) for text in batch['article']]

#             batch['attention_mask'] = [[1] * len(article) + [0] * (512 - len(article)) for article in batch['article']]

#             batch['decoder_input_ids'] = [b.tolist() for b in batch['decoder_input_ids']]
#             batch['decoder_attention_mask'] = [b.tolist() for b in batch['decoder_attention_mask']]
           
#             batch['labels'] = [b.tolist() for b in batch['labels']]

#             batch = {
#                 'input_ids': torch.tensor(batch['article']).to(device),
#                 'attention_mask': torch.tensor(batch['attention_mask']).to(device),
#                 'decoder_input_ids': torch.tensor(batch['decoder_input_ids']).to(device),
#                 'decoder_attention_mask': torch.tensor(batch['decoder_attention_mask']).to(device),
#                 'labels': torch.tensor(batch['labels']).to(device)

#                }
            
#             inputs = {
#                 "input_ids": batch['input_ids'],
#                 "attention_mask": batch['attention_mask'],
#                 "decoder_input_ids": batch['decoder_input_ids'],
#                 "decoder_attention_mask": batch['decoder_attention_mask'],
#                 "labels": batch['labels'],
#             }

#             optimizer.zero_grad()

#             outputs = model(**inputs)
#             loss = outputs.loss
#             loss.backward()

#             optimizer.step()
#             scheduler.step()

In [13]:
# model.eval()

# generated_summaries = []
# target_summaries = []

# with torch.no_grad():
#     for batch in test_loader:
#         inputs = {
#             "input_ids": torch.stack(batch["input_ids"]).to(device),
#             "attention_mask": torch.stack(batch["attention_mask"]).to(device),
#             "decoder_input_ids": torch.stack(batch["decoder_input_ids"]).to(device),
#             "decoder_attention_mask": torch.stack(batch["decoder_attention_mask"]).to(device),
#             "labels": torch.stack(batch["labels"]).to(device),
#         }

#         outputs = model.generate(
#             input_ids=inputs["decoder_input_ids"],
#             attention_mask=inputs["decoder_attention_mask"],
#             max_length=128,
#             num_beams=4,
#             num_return_sequences=1,
#             early_stopping=True,
#         )

#         generated_summaries.extend(tokenizer.batch_decode(outputs, skip_special_tokens=True))
#         target_summaries.extend(tokenizer.batch_decode(inputs["labels"], skip_special_tokens=True))

# # Print the generated and target summaries
# for generated_summary, target_summary in zip(generated_summaries, target_summaries):
#     print("Generated Summary:", generated_summary)
#     print("Target Summary:", target_summary)
#     print()